In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, mean_absolute_error, confusion_matrix, f1_score
from sklearn.feature_selection import SelectKBest, f_classif
import cvxpy as cp
import shap

input_csv_path       = "data_Enhanced_Imputed.csv"
cv_num_folds         = 10
k_features           = 20

gamma_opt = 0.1
C_opt     = 10
theta_delta = -0.4

#boundary index between classes 2 and 3
calib_boundary = 2


#kernel
def rbf_kernel(X1, X2, gamma):
    sq1 = np.sum(X1**2, axis=1)[:, None]
    sq2 = np.sum(X2**2, axis=1)[None, :]
    return np.exp(-gamma * (sq1 + sq2 - 2 * X1.dot(X2.T)))

#SVM dual optimization problem
def solve_dual_svm_qp(X_train, y_train, gamma, C):
    n = len(y_train)
    K = rbf_kernel(X_train, X_train, gamma)
    P_raw = np.outer(y_train, y_train) * K
    P_sym = (P_raw + P_raw.T) / 2.0
    P = cp.psd_wrap(P_sym)
    alpha = cp.Variable(n)
    obj = cp.Maximize(cp.sum(alpha) - 0.5 * cp.quad_form(alpha, P))
    constr = [alpha >= 0, alpha <= C, alpha @ y_train == 0]
    prob = cp.Problem(obj, constr)
    prob.solve(solver=cp.OSQP)
    alpha_opt = alpha.value
    b = 0.0
    for i in range(n):
        if 1e-6 < alpha_opt[i] < C - 1e-6:
            b = y_train[i] - np.sum(alpha_opt * y_train * K[:, i])
            break
    return alpha_opt, b

#boundary models
def prepare_boundary_models(X, y, gamma, C):
    models = {}
    num_classes = np.max(y)
    for k in range(1, num_classes):
        mask = (y <= k) | (y > k)
        Xk, yk = X[mask], np.where(y[mask] > k, 1, -1)
        if len(yk) > 0:
            a_k, b_k = solve_dual_svm_qp(Xk, yk, gamma, C)
            models[k] = (a_k, b_k, Xk, yk)
    return models

#prediction
def build_predictor(models, gamma):
    def predict(Xt):
        preds = []
        for x in Xt:
            f_vals = []
            for k, (a_k, b_k, Xk, yk) in models.items():
                kv = rbf_kernel(Xk, x.reshape(1,-1), gamma).flatten()
                f_vals.append(np.sum(a_k * yk * kv) + b_k)
            preds.append(1 + sum(fv >= 0 for fv in f_vals))
        return np.array(preds)
    return predict

df = pd.read_csv(input_csv_path)
df = df[df['Label_Min']==df['Label_Max']].copy()
feat_cols = [c for c in df.columns if c not in ['SubjectID','Label_Min','Label_Max']]
X_raw = df[feat_cols].values
y = df['Label_Min'].astype(int).values
#imputation & standard scaling
imp = SimpleImputer(strategy='mean')
scaler = StandardScaler()
X_imp = imp.fit_transform(X_raw)
#feature selection
selector = SelectKBest(f_classif, k=k_features)
X_sel = selector.fit_transform(scaler.fit_transform(X_imp), y)
selected_names = selector.get_feature_names_out(feat_cols)

# -- Train final models and apply fixed delta --
models_final = prepare_boundary_models(X_sel, y, gamma_opt, C_opt)
# apply fixed calibration
a2, b2, X2, y2 = models_final[calib_boundary]
models_final[calib_boundary] = (a2, b2 + theta_delta, X2, y2)

#cross-validation
cv = StratifiedKFold(n_splits=cv_num_folds, shuffle=True, random_state=42)
pred_func = build_predictor(models_final, gamma_opt)
accs, maes, f1s = [], [], []
for i, (tr, va) in enumerate(cv.split(X_sel,y), start=1):
    y_pred = pred_func(X_sel[va])
    acc = accuracy_score(y[va], y_pred)
    mae = mean_absolute_error(y[va], y_pred)
    f1 = f1_score(y[va], y_pred, average='weighted')
    print(f"Fold {i}: Accuracy = {acc:.3f}, MAE = {mae:.3f}, F1 = {f1:.3f}")
    accs.append(acc)
    maes.append(mae)
    f1s.append(f1)

print("")
print(f"Overall Accuracy: {np.mean(accs):.3f}")
print(f"Overall MAE: {np.mean(maes):.3f}")
print(f"Overall F1 Score: {np.mean(f1s):.3f}")

#confusion matrix
umax = np.max(y)
cm = confusion_matrix(y, pred_func(X_sel), labels=list(range(1, umax+1)))
cm_df = pd.DataFrame(cm, index=[f'True_{i}' for i in range(1, umax+1)],
                     columns=[f'Pred_{i}' for i in range(1, umax+1)])
print("")
print("Confusion Matrix:")
print(cm_df)

def shap_pred(X):
  return pred_func(X)

#background selection
bg_size = min(100, X_sel.shape[0])
background_idx = np.random.choice(X_sel.shape[0], size=bg_size, replace=False)
background = X_sel[background_idx]

explainer = shap.KernelExplainer(shap_pred, background)

#SHAP values
X_shap = X_sel
shap_values = explainer.shap_values(X_shap, nsamples=100)

#bar chart
shap.summary_plot(
    shap_values,
    X_shap,
    feature_names=selected_names,
    plot_type="bar",
    max_display=5
)

#dot plot
shap.summary_plot(
    shap_values,
    X_shap,
    feature_names=selected_names,
    plot_type="dot",
    max_display=10
)


Fold 1: Accuracy = 0.667, MAE = 0.500, F1 = 0.655
Fold 2: Accuracy = 0.867, MAE = 0.167, F1 = 0.862
Fold 3: Accuracy = 0.833, MAE = 0.267, F1 = 0.831
Fold 4: Accuracy = 0.767, MAE = 0.300, F1 = 0.764
Fold 5: Accuracy = 0.700, MAE = 0.400, F1 = 0.679
Fold 6: Accuracy = 0.633, MAE = 0.600, F1 = 0.595
Fold 7: Accuracy = 0.828, MAE = 0.207, F1 = 0.815
Fold 8: Accuracy = 0.793, MAE = 0.345, F1 = 0.789
Fold 9: Accuracy = 0.759, MAE = 0.448, F1 = 0.745
Fold 10: Accuracy = 0.724, MAE = 0.448, F1 = 0.711

Overall Accuracy: 0.757
Overall MAE: 0.368
Overall F1 Score: 0.745

Confusion Matrix:
        Pred_1  Pred_2  Pred_3  Pred_4
True_1      12       0       0       0
True_2       0     125       3       8
True_3       0      22      28       2
True_4       0      29       8      59


  0%|          | 0/296 [00:00<?, ?it/s]

KeyboardInterrupt: 